In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [205]:
#btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6)])
btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,9)])
G = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))

In [206]:
def included(node, NN_H):
    if node in NN_H:
        return node
    
def get_nodes(graph):
    return graph.map(lambda x: x[0])

def temp(rdd):
    if len(rdd[1][1]) != 0:
        return rdd[1][1][0]
    else:
        return [None]

In [207]:
#Version with broadcasting in the addEdge step on both Min_Selection_Step and Pruning_Step

def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    H = addEdge.groupByKey().map(lambda x: (x[0], set(x[1])))
    return H

def Pruning_Step(H, T):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    v_min = H.map(lambda x: (x[0], min(x[1]))) 
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge2 = NN_H_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    
    #!!!!!!!!!!!!!!!!!!!!!!!!!!
    #If u not in NN: still missing the IsSeed part and solve the Nones here
    H_non = H.filter(lambda x: x[0] not in x[1])
    a = H_non.map(lambda x: (v_min_bc.value[x[0]], x[0])).groupByKey().map(lambda x: (x[0], set(x[1])))
    b = T.cogroup(a).map(lambda x: (x[0], [list(x[1][0])[0], list(x[1][1])]))
    T = b.map(lambda x: (x[0], x[1][0] | set(temp(x))))
    return G, T 

def cracker(G_i):
    
    count = 0
    T = G.map(lambda x: (x[0], set([])))
    
    while G_i.take(1):
        count += 1
        H_i = Min_Selection_Step(G_i)
        G_i, T = Pruning_Step(H_i, T)
        
    return T


In [209]:
cracker(G).collect()

[(0, {1, 2, 3, 5, None}),
 (1, {None}),
 (2, {8, None}),
 (3, {4, 6, 7, None}),
 (4, {None}),
 (5, {None}),
 (6, {None}),
 (7, {None}),
 (8, {None}),
 (9, {10, None}),
 (10, {None})]

In [214]:
H = Min_Selection_Step(G)
print(H.collect())

a = H.filter(lambda x: ( (len(x[1]) <= 1) & (x[0] in x[1])))
a.collect()

#We have to remove those that are not active, which are 0 and 3
#Either include this info in the value of each node 
#or maybe taking those that are not in the result of former two phases??

[(0, {0}), (8, {2, 3, 5}), (4, {3}), (1, {0, 1}), (5, {1, 2, 5}), (9, {9}), (2, {0, 1, 2}), (10, {9}), (6, {3}), (7, {3, 5}), (3, {3})]


[(0, {0}), (9, {9}), (3, {3})]